In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
final_data = pd.read_excel("D:\\Documents\\LUMS\\7th Semester Courses\\SPROJ\\final_data.xlsx")

columns_to_remove = ['mkt_structure_fibre_fullcomp', 'mkt_structure_fixed_wireless_broadband_fullcomp','region_Asia','income_group_low_income','received_wages_pct','borrowed_for_health_pct','emergency_funds_possible_somewhat_diff_pct','emergency_funds_possible_notdiff_pct','emergency_funds_possible_pct']
final_data = final_data.drop(columns=columns_to_remove)

In [11]:
# mean and standard deviation

filtered_columns = [col for col in final_data.columns if final_data.loc[0, col] in [2,3, 4]]

# Calculate mean and standard deviation for these columns excluding row 0
result = []
for col in filtered_columns:
    mean_value = final_data.loc[1:, col].mean()  # Exclude row 0
    std_dev = final_data.loc[1:, col].std()     # Exclude row 0
    result.append((col, mean_value, std_dev))

# Display results
for col_name, mean, std in result:
    print(f"Column: {col_name}, Mean: {mean:.2f}, Std Dev: {std:.2f}")

Column: skill_internet_banking_pct, Mean: 29.28, Std Dev: 28.34
Column: skill_uploading_content_pct, Mean: 20.23, Std Dev: 13.95
Column: skill_arithmetic_formula_spreadsheet_pct, Mean: 23.98, Std Dev: 16.47
Column: skill_software_run_over_the_internet_pct, Mean: 18.67, Std Dev: 17.04
Column: skill_programming_pct, Mean: 5.22, Std Dev: 4.37
Column: borrowed_from_formal_pct, Mean: 24.49, Std Dev: 19.03
Column: used_phone_or_internet_send_money_pct, Mean: 32.90, Std Dev: 20.64
Column: saved_fin_account_or_mobile_pct, Mean: 31.50, Std Dev: 20.66
Column: deposit_has_fin_account_pct, Mean: 64.32, Std Dev: 21.17
Column: not_worried_index, Mean: 8.12, Std Dev: 4.57
Column: somewhat_worried_index, Mean: 7.31, Std Dev: 1.31
Column: very_worried_index, Mean: 9.28, Std Dev: 5.19
Column: worried_index, Mean: 5.48, Std Dev: 0.92
Column: borrowed_any_pct, Mean: 50.15, Std Dev: 12.90


# Table of Contents
1. [OLS](#OLS)
    - [Direct Link Regression without controls](#Direct-Link-Regression-without-controls)
    - [Direct Link Regression with controls](#direct-link-regression-with-controls)
    - [Indirect Link Regression without controls](#indirect-link-regression-without-controls)
    - [Indirect Link Regression with controls](#indirect-link-regression-with-controls)
2. [Random Forest](#random_forest)
    - [Random Forest without controls](#random-forest-without-controls)
    - [Random Forest with controls](#random-forest-with-controls)

# OLS

##  Direct Link Regression without controls

In [12]:
import pandas as pd
import statsmodels.api as sm

# Assuming final_data is already loaded and available

# Define dependent variables based on your requirement
y_vars = ['worried_index', 'very_worried_index', 'somewhat_worried_index', 'not_worried_index','borrowed_any_pct','deposit_has_fin_account_pct']

# Define a new output file name
output_file = "Direct_Link_Regression_without_controls.txt"

with open(output_file, "w") as f:
    # Loop through each dependent variable (y_var)
    for y_var in y_vars:
        f.write("=" * 80 + "\n")
        f.write(f"OLS Regression for Dependent Variable: {y_var}\n")
        f.write("=" * 80 + "\n\n")

        # Define independent variables (x_vars) only
        x_vars = final_data.columns[final_data.iloc[0] == 2].tolist()  # Columns where row 0 equals 2
        y_variable = y_var

        # Remove chain identification (skip the first row)
        working_data = final_data.iloc[1:]

        # Define X (independent variables) and Y (dependent variable)
        X = working_data[x_vars]
        Y = working_data[y_variable]

        # Add a constant (intercept) to the independent variables
        X = sm.add_constant(X)

        # Fit the OLS regression model
        model = sm.OLS(Y, X).fit()

        # Capture the original summary as text
        original_summary = model.summary().as_text()

        # Extract the coefficient table lines
        lines = original_summary.split("\n")
        start_index = next(i for i, line in enumerate(lines) if "coef" in line)  # Start of coefficient table
        end_index = next(i for i, line in enumerate(lines[start_index:]) if "---" in line) + start_index  # End of table

        # Include the header row from the coefficient table
        coefficient_header = lines[start_index]  # Header row is the start_index line
        coefficient_lines = lines[start_index + 1:end_index + 1]
        
        # Filter rows to include only `x_vars` and the constant
        filtered_lines = [
            line for line in coefficient_lines if any(var in line for var in x_vars) or "const" in line
        ]

        # Reconstruct the filtered summary
        filtered_summary = "\n".join(lines[:start_index] + [coefficient_header] + filtered_lines + lines[end_index + 1:])

        # Print filtered results to console
        print(f"OLS Regression Results for Dependent Variable: {y_var}")
        print(filtered_summary)
        print("\n")

        # Write filtered results to the text file
        f.write(f"Filtered Results for Dependent Variable: {y_var}\n")
        f.write(filtered_summary)
        f.write("\n\n" + "=" * 80 + "\n\n")

# Print a confirmation message
print(f"Filtered OLS regression results have been saved to '{output_file}'.")

OLS Regression Results for Dependent Variable: worried_index
                            OLS Regression Results                            
Dep. Variable:          worried_index   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     11.51
Date:                Fri, 27 Dec 2024   Prob (F-statistic):           2.73e-08
Time:                        03:03:41   Log-Likelihood:                -85.789
No. Observations:                  82   AIC:                             183.6
Df Residuals:                      76   BIC:                             198.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
const                                   

## Direct Link Regression with controls

In [13]:
import pandas as pd
import statsmodels.api as sm

# Define y_variables based on a condition (in this case, where row 0 value is 4)
y_vars = final_data.columns[final_data.iloc[0] == 4].tolist()

# Open the text file to save the results
output_file = "Direct_Link_Results_with_controls.txt"
with open(output_file, "w") as f:
    # Loop through each dependent variable (y_var)
    for y_var in y_vars:
        f.write("=" * 80 + "\n")
        f.write(f"OLS Regression for Dependent Variable: {y_var}\n")
        f.write("=" * 80 + "\n\n")

        # Define independent variables (x_vars) and control variables
        x_vars = final_data.columns[final_data.iloc[0] == 2].tolist()
        control_variables = final_data.columns[final_data.iloc[0] == 1].tolist()
        y_variable = y_var

        # Combine X and control variables into one predictor set
        predictors = x_vars + control_variables

        # Remove chain identification (skip the first row)
        working_data = final_data.iloc[1:]

        # Define X (independent variables) and Y (dependent variable)
        X = working_data[predictors]
        Y = working_data[y_variable]

        # Add a constant (intercept) to the independent variables
        X = sm.add_constant(X)

        # Fit the OLS regression model
        model = sm.OLS(Y, X).fit()

        # Print and write the model summary in the text file
        console_output = model.summary().as_text()
        print(console_output)  # Print to console

        f.write(console_output)  # Write to the text file
        f.write("\n\n" + "=" * 80 + "\n\n")

# Print a confirmation message
print(f"OLS regression results have been saved to '{output_file}'.")


                                 OLS Regression Results                                
Dep. Variable:     deposit_has_fin_account_pct   R-squared:                       0.914
Model:                                     OLS   Adj. R-squared:                  0.827
Method:                          Least Squares   F-statistic:                     10.42
Date:                         Fri, 27 Dec 2024   Prob (F-statistic):           8.34e-12
Time:                                 03:03:41   Log-Likelihood:                -265.38
No. Observations:                           82   AIC:                             614.8
Df Residuals:                               40   BIC:                             715.8
Df Model:                                   41                                         
Covariance Type:                     nonrobust                                         
                                                         coef    std err          t      P>|t|      [0.025      0.975]
-

## Indirect Link Regression without controls

In [14]:
import pandas as pd
import statsmodels.api as sm

# Assuming final_data is already loaded and available

# Define dependent variables based on your requirement
y_vars = ['borrowed_from_formal_pct','used_phone_or_internet_send_money_pct','saved_fin_account_or_mobile_pct']

# Define a new output file name
output_file = "Indirect_Link_Regression_without_controls.txt"

with open(output_file, "w") as f:
    # Loop through each dependent variable (y_var)
    for y_var in y_vars:
        f.write("=" * 80 + "\n")
        f.write(f"OLS Regression for Dependent Variable: {y_var}\n")
        f.write("=" * 80 + "\n\n")

        # Define independent variables (x_vars) only
        x_vars = final_data.columns[final_data.iloc[0] == 2].tolist()  # Columns where row 0 equals 2
        y_variable = y_var

        # Remove chain identification (skip the first row)
        working_data = final_data.iloc[1:]

        # Define X (independent variables) and Y (dependent variable)
        X = working_data[x_vars]
        Y = working_data[y_variable]

        # Add a constant (intercept) to the independent variables
        X = sm.add_constant(X)

        # Fit the OLS regression model
        model = sm.OLS(Y, X).fit()

        # Capture the original summary as text
        original_summary = model.summary().as_text()

        # Extract the coefficient table lines
        lines = original_summary.split("\n")
        start_index = next(i for i, line in enumerate(lines) if "coef" in line)  # Start of coefficient table
        end_index = next(i for i, line in enumerate(lines[start_index:]) if "---" in line) + start_index  # End of table

        # Include the header row from the coefficient table
        coefficient_header = lines[start_index]  # Header row is the start_index line
        coefficient_lines = lines[start_index + 1:end_index + 1]
        
        # Filter rows to include only `x_vars` and the constant
        filtered_lines = [
            line for line in coefficient_lines if any(var in line for var in x_vars) or "const" in line
        ]

        # Reconstruct the filtered summary
        filtered_summary = "\n".join(lines[:start_index] + [coefficient_header] + filtered_lines + lines[end_index + 1:])

        # Print filtered results to console
        print(f"OLS Regression Results for Intermediate Variable: {y_var}")
        print(filtered_summary)
        print("\n")

        # Write filtered results to the text file
        f.write(f"Filtered Results for Intermediate Variable: {y_var}\n")
        f.write(filtered_summary)
        f.write("\n\n" + "=" * 80 + "\n\n")

# Print a confirmation message
print(f"Filtered OLS regression results have been saved to '{output_file}'.")


OLS Regression Results for Intermediate Variable: borrowed_from_formal_pct
                               OLS Regression Results                               
Dep. Variable:     borrowed_from_formal_pct   R-squared:                       0.688
Model:                                  OLS   Adj. R-squared:                  0.667
Method:                       Least Squares   F-statistic:                     33.45
Date:                      Fri, 27 Dec 2024   Prob (F-statistic):           6.81e-18
Time:                              03:03:41   Log-Likelihood:                -309.74
No. Observations:                        82   AIC:                             631.5
Df Residuals:                            76   BIC:                             645.9
Df Model:                                 5                                         
Covariance Type:                  nonrobust                                         
                                               coef    std err          t  

## Indirect Link Regression with controls

In [15]:
import pandas as pd
import statsmodels.api as sm

# Assuming final_data is already loaded and available

# Define dependent variables based on your requirement
y_vars = ['borrowed_from_formal_pct', 'used_phone_or_internet_send_money_pct', 'saved_fin_account_or_mobile_pct']

# Define a new output file name
output_file = "Indirect_Link_Regression_with_controls.txt"

# Initialize a text file writer
with open(output_file, "w") as file:
    # Loop through each dependent variable (y_var)
    for y_var in y_vars:
        # Define independent variables (x_vars) only
        x_vars = final_data.columns[final_data.iloc[0] == 2].tolist()  # Columns where row 0 equals 2
        y_variable = y_var
        control_variables = final_data.columns[final_data.iloc[0] == 1].tolist()

        # Combine predictors and prepare data
        predictors = x_vars + control_variables

        # Remove chain identification (skip the first row)
        working_data = final_data.iloc[1:]

        # Define X (independent variables) and Y (dependent variable)
        X = working_data[predictors]
        Y = working_data[y_variable]

        # Add a constant (intercept) to the independent variables
        X = sm.add_constant(X)

        # Fit the OLS regression model
        model = sm.OLS(Y, X).fit()

        # Write regression summary to the text file
        file.write(f"Results for Dependent Variable: {y_var}\n")
        file.write(model.summary().as_text())
        file.write("\n" + "="*80 + "\n\n")

# Print a confirmation message
print(f"Filtered OLS regression results have been saved to '{output_file}'.")

Filtered OLS regression results have been saved to 'Indirect_Link_Regression_with_controls.txt'.


# Random Forest

## Random Forest without controls

In [16]:
# Specify the target variable
# Open a text file for writing results
with open("Random_Forest_without_controls.txt", "w") as file:
    y_variables = final_data.columns[final_data.iloc[0] == 4].tolist()

    for y_var in y_variables:
        # Select key independent variables and control variables for features
        independent_variables = final_data.columns[final_data.iloc[0] == 2].tolist()
        
        features = independent_variables

        working_data = final_data.iloc[1:]

        # Create feature and target datasets
        X = working_data[features]
        y = working_data[y_var]

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the Random Forest model
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)

        # Make predictions
        y_pred = rf.predict(X_test)

        # Evaluate the model
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Write evaluation metrics to the file
        file.write(f"Results for Target Variable: {y_var}\n")
        file.write(f"Mean Absolute Error: {mae:.4f}\n")
        file.write(f"Mean Squared Error: {mse:.4f}\n")
        file.write(f"R² Score: {r2:.4f}\n\n")

        # Analyze feature importance
        feature_importances = pd.DataFrame({
            'Feature': features,
            'Importance': rf.feature_importances_
        }).sort_values(by='Importance', ascending=False)

        # Write feature importances to the file
        file.write("Feature Importances:\n")
        for _, row in feature_importances.iterrows():
            file.write(f"  {row['Feature']}: {row['Importance']:.4f}\n")
        file.write("\n" + "-"*50 + "\n\n")



## Random Forest with controls

In [17]:
# Specify the target variable
# Open a text file for writing results
with open("Random_Forest_with_controls.txt", "w") as file:
    y_variables = final_data.columns[final_data.iloc[0] == 4].tolist()

    for y_var in y_variables:
        # Select key independent variables and control variables for features
        control_variables = final_data.columns[final_data.iloc[0] == 1].tolist()
        independent_variables = final_data.columns[final_data.iloc[0] == 2].tolist()

        # Combine independent variables and control variables
        features = independent_variables + control_variables

        working_data = final_data.iloc[1:]

        # Create feature and target datasets
        X = working_data[features]
        y = working_data[y_var]

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the Random Forest model
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)

        # Make predictions
        y_pred = rf.predict(X_test)

        # Evaluate the model
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Write evaluation metrics to the file
        file.write(f"Results for Target Variable: {y_var}\n")
        file.write(f"Mean Absolute Error: {mae:.4f}\n")
        file.write(f"Mean Squared Error: {mse:.4f}\n")
        file.write(f"R² Score: {r2:.4f}\n\n")

        # Analyze feature importance
        feature_importances = pd.DataFrame({
            'Feature': features,
            'Importance': rf.feature_importances_
        }).sort_values(by='Importance', ascending=False)

        # Write feature importances to the file
        file.write("Feature Importances:\n")
        for _, row in feature_importances.iterrows():
            file.write(f"  {row['Feature']}: {row['Importance']:.4f}\n")
        file.write("\n" + "-"*50 + "\n\n")